# T3D (Transform 3D module)
## A transform 3D module for Python

In [2]:
from math import cos, sin, tan

import numpy as np
import sympy as sp
sp.init_printing()

from nbsupport import md, _subs

Symbolic = {}

# 基本的なユーティリティ関数

In [3]:
def __M32__(ARGS, EXPR):
    f = sp.lambdify(ARGS, EXPR, 'numpy')
    return lambda *args: np.array(f(*args), dtype=np.float32)

def __V32__(ARGS, EXPR):
    f = __M32__(ARGS, EXPR)
    return lambda *args: f(*args).reshape((len(ARGS),))

Symbolic['M32'] = __M32__
Symbolic['V32'] = __V32__

## ベクトルの扱い

ベクトルの定義

In [4]:
# 数式としてのベクトルの雛形の定義
sp.var('x y z w')

VEC2 = sp.Matrix([x, y])
VEC3 = sp.Matrix([x, y, z])
VEC4 = sp.Matrix([x, y, z, w])
def VEC(name, n): return sp.MatrixSymbol(name, n, 1)

def Vec2(X, Y):       return _subs(VEC2, {x: X, y: Y})
def Vec3(X, Y, Z):    return _subs(VEC3, {x: X, y: Y, z: Z})
def Vec4(X, Y, Z, W): return _subs(VEC4, {x: X, y: Y, z: Z, w: W})
def Vec(n, name):     return sp.MatrixSymbol(sp.Symbol(name), n, 1)

Symbolic['vec2'] = Symbolic['V32']((x, y), VEC2)
Symbolic['vec3'] = Symbolic['V32']((x, y, z), VEC3)
Symbolic['vec4'] = Symbolic['V32']((x, y, z, w), VEC4)

# ベクトルの正規化
def normalize(v):
    norm=np.linalg.norm(v)
    if norm==0: return v
    return v/norm

## 同次座標系 (Homogeneous coordinate system)


三次元デカルト座標系(*Cartesian coordinate system*)における点の座標$(x, y, z)$の**同次座標**(*Homogeneous coordinate*)は，任意の非零実数$w$を用いて，$(wx, wy, wz, w)$で表します．

逆に，同次座標$(x, y, z, w)$のデカルト座標は$(x/w, y/w, z/w)$で与えられます．同次座標を用いた座標系のことを**同次座標系**(*Homogeneous coordinate system*)と呼びます．

In [5]:
def Homogeneous(p, w=1):
    return (p * w).col_join(sp.Matrix([w]))

def Cartesian(h):
    return h[0:3,-1] / h[3,0]

if __name__ == '__main__':
    p = Vec3(x, y, z)
    assert sp.Eq(p, Cartesian(Homogeneous(p, w=w)))

$e_1, e_2, e_3 \mapsto e_1', e_2', e_3'$に写す写像$T$について考えてみよう．

$$\begin {align}
  T\begin {pmatrix}e_1 & e_2 & e_3\end {pmatrix} &= \begin {pmatrix}e_1' & e_2' & e_3'\end {pmatrix} \\
  T &= \begin {pmatrix}e_1' & e_2' & e_3'\end {pmatrix} \cdot \begin {pmatrix}e_1 & e_2 & e_3\end {pmatrix}^{-1}
\end {align}$$

## 行列

以下は行列の構成子です．Pythonのライブラリ関数の実装に用いています．

In [6]:
# すべてのAPIをSymbolicに記述したら不要になる

def mat4x4(m00, m01, m02, m03, m10, m11, m12, m13,
        m20, m21, m22, m23, m30, m31, m32, m33):
    return np.array([[m00, m01, m02, m03], [m10, m11, m12, m13],
        [m20, m21, m22, m23], [m30, m31, m32, m33]], dtype=np.float32)

def tmat4x4(m00, m01, m02, m03, m10, m11, m12, m13,
        m20, m21, m22, m23, m30, m31, m32, m33):
    return mat4x4(m00, m01, m02, m03, m10, m11, m12, m13,
        m20, m21, m22, m23, m30, m31, m32, m33).transpose()

if __name__ == '__main__':
    print('行列の構成子のテスト')
    print('mat4x4(1, 2, ..., 16):\n{0}'.format(mat4x4(*range(1, 17))))
    print('tmat4x4(1, 2, ..., 16):\n{0}'.format(tmat4x4(*range(1, 17))))

行列の構成子のテスト
mat4x4(1, 2, ..., 16):
[[  1.   2.   3.   4.]
 [  5.   6.   7.   8.]
 [  9.  10.  11.  12.]
 [ 13.  14.  15.  16.]]
tmat4x4(1, 2, ..., 16):
[[  1.   5.   9.  13.]
 [  2.   6.  10.  14.]
 [  3.   7.  11.  15.]
 [  4.   8.  12.  16.]]


## 拡大縮小変換 (Scale)

拡大縮小変換行列$\mathrm {Scale}(s_x, s_y, s_z)$に同次座標を乗ずると，その座標の$X$-, $Y$-, $Z$-成分をそれぞれ$s_x, s_y, s_z$倍した座標を与えます．

In [7]:
ScaleSymbols = sp.var('s_x s_y s_z')
Symbolic['Scale'] = sp.diag(s_x, s_y, s_z, 1)
Symbolic['scale'] = Symbolic['M32'](ScaleSymbols, Symbolic['Scale'])

# Todo: もしかして、Scale(s_x, s_y, s_z) => Scale行列の方がいいかも

if __name__ == '__main__':
    md('#### 拡大縮小行列: $\mathrm {Scale}(s_x, s_y, s_z)$')
    p = Vec3(x, y, z)
    md('$$\mathrm {Scale}(s_x, s_y, s_z)', Homogeneous(p), '=',
       Symbolic['Scale'], Homogeneous(p), '=',
       (Symbolic['Scale'] * Homogeneous(p)), '$$')

#### 拡大縮小行列: $\mathrm {Scale}(s_x, s_y, s_z)$

$$\mathrm {Scale}(s_x, s_y, s_z)\left[\begin{matrix}x\\y\\z\\1\end{matrix}\right]=\left[\begin{matrix}s_{x} & 0 & 0 & 0\\0 & s_{y} & 0 & 0\\0 & 0 & s_{z} & 0\\0 & 0 & 0 & 1\end{matrix}\right]\left[\begin{matrix}x\\y\\z\\1\end{matrix}\right]=\left[\begin{matrix}s_{x} x\\s_{y} y\\s_{z} z\\1\end{matrix}\right]$$

## 回転変換: Rotate

回転変換行列($\mathrm {Rotate}_X(\theta)$に同次座標を乗ずると，その座標を$X$軸のまわりに$\theta$だけ回転した点の同次座標を与えます．ほかの軸に対する回転行列の働きも同様です．

In [8]:
sp.var('theta')
c, s = sp.cos(theta), sp.sin(theta)

Symbolic['RotateX'] = sp.diag(1, sp.Matrix(2, 2, [ c, s, -s, c]), 1)
Symbolic['RotateY'] = sp.diag(sp.Matrix(3, 3, [ c, 0, -s, 0, 1, 0, s, 0, c ]), 1)
Symbolic['RotateZ'] = sp.diag(sp.Matrix(2, 2, [ c, s, -s, c ]), 1, 1)

# Todo: もしかして、RotateX(theta) => Rotate行列の方がいいかも

Symbolic['rotateX'] = Symbolic['M32'](theta, Symbolic['RotateX'])
Symbolic['rotateY'] = Symbolic['M32'](theta, Symbolic['RotateY'])
Symbolic['rotateZ'] = Symbolic['M32'](theta, Symbolic['RotateZ'])

if __name__ == '__main__':
    for R in 'X Y Z'.split():
        md('#### $\mathit{Rotate}_', R, '(\\theta)$: ', R, '軸を中心に回転$$', Symbolic['Rotate' + R], '$$')
    
    md('#### X, Y, Z軸を中心に45度回転')
    md('$$', tuple([Cartesian(Symbolic[R].subs({ theta: sp.pi / 4 }) *
                             Homogeneous(Vec3(x, y, z)))
                   for R in 'RotateX RotateY RotateZ'.split()]), '$$')

#### $\mathit{Rotate}_X(\theta)$: X軸を中心に回転$$\left[\begin{matrix}1 & 0 & 0 & 0\\0 & \cos{\left (\theta \right )} & \sin{\left (\theta \right )} & 0\\0 & - \sin{\left (\theta \right )} & \cos{\left (\theta \right )} & 0\\0 & 0 & 0 & 1\end{matrix}\right]$$

#### $\mathit{Rotate}_Y(\theta)$: Y軸を中心に回転$$\left[\begin{matrix}\cos{\left (\theta \right )} & 0 & - \sin{\left (\theta \right )} & 0\\0 & 1 & 0 & 0\\\sin{\left (\theta \right )} & 0 & \cos{\left (\theta \right )} & 0\\0 & 0 & 0 & 1\end{matrix}\right]$$

#### $\mathit{Rotate}_Z(\theta)$: Z軸を中心に回転$$\left[\begin{matrix}\cos{\left (\theta \right )} & \sin{\left (\theta \right )} & 0 & 0\\- \sin{\left (\theta \right )} & \cos{\left (\theta \right )} & 0 & 0\\0 & 0 & 1 & 0\\0 & 0 & 0 & 1\end{matrix}\right]$$

#### X, Y, Z軸を中心に45度回転

$$\left ( \left[\begin{matrix}x\\\frac{\sqrt{2} y}{2} + \frac{\sqrt{2} z}{2}\\- \frac{\sqrt{2} y}{2} + \frac{\sqrt{2} z}{2}\end{matrix}\right], \quad \left[\begin{matrix}\frac{\sqrt{2} x}{2} - \frac{\sqrt{2} z}{2}\\y\\\frac{\sqrt{2} x}{2} + \frac{\sqrt{2} z}{2}\end{matrix}\right], \quad \left[\begin{matrix}\frac{\sqrt{2} x}{2} + \frac{\sqrt{2} y}{2}\\- \frac{\sqrt{2} x}{2} + \frac{\sqrt{2} y}{2}\\z\end{matrix}\right]\right )$$

## 並行移動変換 (Translate)

並行移動変換行列$\mathrm {Translate}(t_x, t_y, t_z)$に同次座標を乗ずると，その座標を$X, Y, Z$軸方向にそれぞれ$t_x, t_y, t_z$だけ並行移動した点の同次座標を与えます．

In [9]:
[tx, ty, tz] = TranslateSymbols = sp.symbols('t_x t_y t_z')
Symbolic['Translate'] = sp.eye(3).row_join(sp.Matrix(TranslateSymbols)).col_join(sp.Matrix([[0, 0, 0, 1]]))
Symbolic['translate'] = Symbolic['M32'](TranslateSymbols, Symbolic['Translate'])

# Todo: もしかして Translate(t_x, t_y, t_z) => Translate行列の方がいいかも

if __name__ == '__main__':
    p = Homogeneous(Vec3(x, y, z))

    md('#### 並行移動行列')
    md('$\mathrm {Translate}(t_x, t_y, t_z)', p, '=',
       Symbolic['Translate'], p, '=',
       Symbolic['Translate'] * p, '$')

#### 並行移動行列

$\mathrm {Translate}(t_x, t_y, t_z)\left[\begin{matrix}x\\y\\z\\1\end{matrix}\right]=\left[\begin{matrix}1 & 0 & 0 & t_{x}\\0 & 1 & 0 & t_{y}\\0 & 0 & 1 & t_{z}\\0 & 0 & 0 & 1\end{matrix}\right]\left[\begin{matrix}x\\y\\z\\1\end{matrix}\right]=\left[\begin{matrix}t_{x} + x\\t_{y} + y\\t_{z} + z\\1\end{matrix}\right]$

## 空間変換のための関数群

### 視野変換 (LookAt)

視野変換は**全体座標系** (*Global coordinate system*)に配置されたオブジェクトを観察者の立場から眺めたときの様子，すなわち**視野座標系**(*Viewing coordinate system*)に変換します．観察者の立ち位置を表現するために観察者の視点(*eye*)，観察者の視線の先の点(*center*)，そして観察者の頭の向き(*up*)を与えます．

視野変換は全体座標系を視点に平行移動する変換$T$と，視線の向きを$Z$軸方向から視線の向きに回転する変換$R$を合成($R \cdot T$)したものと考えられます．

まず，平行移動は原点を移転に移動する変換で，特に$\mathit {eye}$を原点に移動する移動です．移動のベクトルは$(0 - \mathit {eye}) = -\mathit {eye}$です．

次に回転変換$R$ですが，視野座標系を定める3つの軸は，視線の向き(*Forward*)，頭の向き(*Head*)，そして両者に直交する横方向の向き(*Side*)で構成できそうなのですが，一般には視線の向きと頭の向きが直交しているとは限りません．そこで，3軸は以下のように取ります．

- 視線の向き ($F$): $(\mathit {center} - \mathit {eye})$
- 横の向き ($S$): 「視線の向き」と「頭の向き」に直交するベクトル
- 上の向き ($H$): 「横の向き」と「視線の向き」に直交するベクトル

すなわち，$F = \mathit {normalize}(\mathit {center} - \mathit {eye}) \qquad
  S = \mathit {normalize}(F \times \mathit {up}) \qquad
  H = S \times F$です．
  
回転行列$R$は，$S, H, F$をそれぞれ$X, Y, Z$軸方向の単位ベクトル($e_X, e_Y, e_Z$)軸に射影します．さて，逆に$R$の逆回転$R^{-1}$は$(e_x, e_y, e_z)$をそれぞれ$S, H, F$に射影する行列となので簡単に求まります．

目的とする行列$R$はこの回転行列の逆行列なのですが、回転行列は正規直交基底を構成するので，その逆行列は，その行列の転置行列$R = (R^{-1})^T$となります．

視野変換行列$\mathit {LookAt}$は，$T$と$R$の合成として$\mathrm {LookAt} = R \cdot T$と与えることができます．

ただし，CGやOpenGLでは，全体座標系に右手系を用いる一方，視野座標系には左手系を用いることが一般的です．このため，以下のPythonでの実装では，`F = -F`として右手系から左手系への変換を実施しています．

In [10]:
LookAtVectors = [Eye, Side, Head, Forward] = [
    sp.var('I_x I_y I_z'), sp.var('S_x S_y S_z'),
    sp.var('H_x H_y H_z'), sp.var('F_x F_y F_z')]
[I, S, H, F] = [sp.Matrix(3, 1, M) for M in LookAtVectors]

LookAtTranslate = Symbolic['Translate'].subs({ tx: -I_x, ty: -I_y, tz: -I_z })
invLookAtRotate = sp.Matrix(sp.BlockMatrix([[S, H, -F, sp.zeros(3, 1)]])).col_join(sp.Matrix([[0, 0, 0, 1]]))
LookAtRotate = invLookAtRotate.T  # 回転行列は正規直交行列なので，逆行列は転置行列

Symbolic['LookAt'] = LookAtRotate * LookAtTranslate

__lookat__ = Symbolic['M32'](Eye + Forward + Side + Head, Symbolic['LookAt'])
def __lookAtAux__(eye, center, up):
    i = eye
    f = normalize(center - eye)
    s = normalize(np.cross(f, up))
    h = np.cross(s, f)
    
    # もしかして __lookat__(*i, *f, *s, *h) でいける？
    
    return __lookat__(i[0], i[1], i[2],
                      f[0], f[1], f[2],
                      s[0], s[1], s[2],
                      h[0], h[1], h[2])
Symbolic['lookat'] = __lookAtAux__

if __name__ == '__main__':
    md('#### 視点へのカメラの移動 (LookAtTranslate)\n',
       'まず，視点($I$)を原点に移動する平行変換です．',
       '$$\\mathrm {LookAtTranslate} = \\mathrm {Translate}(-I) =',
       LookAtTranslate, '$$')
    
    md('この変換が視点を原点に移動することを確認してみましょう．$$',
       '\\mathrm {LookAtTranslate} \\cdot', Homogeneous(I), '=',
       LookAtTranslate, '\\cdot', Homogeneous(I), '=',
       LookAtTranslate * Homogeneous(I), '$$')

    md('#### 視線の設定 (LookAtRotate)\n',
       'つぎに，逆回転変換は標準的な基底を$S, H, -F$に移動します．',
       'ここで$-F$と反転しているのは，全体座標系は右手系で与え，視野座標系は左手系で与える慣習に沿ったためです．',
       '$$',
       '\\mathrm {LookAtRotate}^{-1} = ', invLookAtRotate, '$$',
       'では，この逆回転変換が実際に基底ベクトルを$S, H, F$に写すか確認してみましょう．$$\\left(',
       Vec3(1, 0, 0), '\mapsto', Cartesian(invLookAtRotate * Homogeneous(Vec3(1, 0, 0))), ',',
       Vec3(0, 1, 0), '\mapsto', Cartesian(invLookAtRotate * Homogeneous(Vec3(0, 1, 0))), ',',
       Vec3(0, 0, 1), '\mapsto', Cartesian(invLookAtRotate * Homogeneous(Vec3(0, 0, 1))),
       '\\right)$$')
    md('| $S$ | $H$ | $-F$ |\n',
       '|-----|-----|-----|\n',
       '|$', Vec3(1, 0, 0), '\mapsto', Cartesian(invLookAtRotate * Homogeneous(Vec3(1, 0, 0))), '$',
       '|$', Vec3(1, 0, 0), '\mapsto', Cartesian(invLookAtRotate * Homogeneous(Vec3(1, 0, 0))), '$',
       '|$', Vec3(1, 0, 0), '\mapsto', Cartesian(invLookAtRotate * Homogeneous(Vec3(1, 0, 0))), '$ |')
    md('逆回転変換の逆行列が求めたい回転変換なのです．$S, H, F$は標準化され，相互に直交しています．',
       'したがって，この変換は正規直交変換ということになります．',
       '正規直交変換の逆行列は転置行列になるので，視野変換の回転行列は以下のようになります．$$',
       '\\mathrm {LookAtRotate} =',
       '(\\mathrm {LookAtRotate}^{-1})^{-1} =',
       '(\\mathrm {LookAtRotate}^{-1})^T =',
       invLookAtRotate, '^T =', LookAtRotate, '$$')

    md('#### 視野変換 (LookAt行列)')
    md('平行移動行列と回転行列を合成することで視野変換行列が得られます．$$',
       '\\mathrm {LookAt} = \\mathrm {LookAtRotate} \\cdot \\mathrm {LookAtTranslate} =',
       Symbolic['LookAt'], '$$')

#### 視点へのカメラの移動 (LookAtTranslate)
まず，視点($I$)を原点に移動する平行変換です．$$\mathrm {LookAtTranslate} = \mathrm {Translate}(-I) =\left[\begin{matrix}1 & 0 & 0 & - I_{x}\\0 & 1 & 0 & - I_{y}\\0 & 0 & 1 & - I_{z}\\0 & 0 & 0 & 1\end{matrix}\right]$$

この変換が視点を原点に移動することを確認してみましょう．$$\mathrm {LookAtTranslate} \cdot\left[\begin{matrix}I_{x}\\I_{y}\\I_{z}\\1\end{matrix}\right]=\left[\begin{matrix}1 & 0 & 0 & - I_{x}\\0 & 1 & 0 & - I_{y}\\0 & 0 & 1 & - I_{z}\\0 & 0 & 0 & 1\end{matrix}\right]\cdot\left[\begin{matrix}I_{x}\\I_{y}\\I_{z}\\1\end{matrix}\right]=\left[\begin{matrix}0\\0\\0\\1\end{matrix}\right]$$

#### 視線の設定 (LookAtRotate)
つぎに，逆回転変換は標準的な基底を$S, H, -F$に移動します．ここで$-F$と反転しているのは，全体座標系は右手系で与え，視野座標系は左手系で与える慣習に沿ったためです．$$\mathrm {LookAtRotate}^{-1} = \left[\begin{matrix}S_{x} & H_{x} & - F_{x} & 0\\S_{y} & H_{y} & - F_{y} & 0\\S_{z} & H_{z} & - F_{z} & 0\\0 & 0 & 0 & 1\end{matrix}\right]$$では，この逆回転変換が実際に基底ベクトルを$S, H, F$に写すか確認してみましょう．$$\left(\left[\begin{matrix}1\\0\\0\end{matrix}\right]\mapsto\left[\begin{matrix}S_{x}\\S_{y}\\S_{z}\end{matrix}\right],\left[\begin{matrix}0\\1\\0\end{matrix}\right]\mapsto\left[\begin{matrix}H_{x}\\H_{y}\\H_{z}\end{matrix}\right],\left[\begin{matrix}0\\0\\1\end{matrix}\right]\mapsto\left[\begin{matrix}- F_{x}\\- F_{y}\\- F_{z}\end{matrix}\right]\right)$$

| $S$ | $H$ | $-F$ |
|-----|-----|-----|
|$\left[\begin{matrix}1\\0\\0\end{matrix}\right]\mapsto\left[\begin{matrix}S_{x}\\S_{y}\\S_{z}\end{matrix}\right]$|$\left[\begin{matrix}1\\0\\0\end{matrix}\right]\mapsto\left[\begin{matrix}S_{x}\\S_{y}\\S_{z}\end{matrix}\right]$|$\left[\begin{matrix}1\\0\\0\end{matrix}\right]\mapsto\left[\begin{matrix}S_{x}\\S_{y}\\S_{z}\end{matrix}\right]$ |

逆回転変換の逆行列が求めたい回転変換なのです．$S, H, F$は標準化され，相互に直交しています．したがって，この変換は正規直交変換ということになります．正規直交変換の逆行列は転置行列になるので，視野変換の回転行列は以下のようになります．$$\mathrm {LookAtRotate} =(\mathrm {LookAtRotate}^{-1})^{-1} =(\mathrm {LookAtRotate}^{-1})^T =\left[\begin{matrix}S_{x} & H_{x} & - F_{x} & 0\\S_{y} & H_{y} & - F_{y} & 0\\S_{z} & H_{z} & - F_{z} & 0\\0 & 0 & 0 & 1\end{matrix}\right]^T =\left[\begin{matrix}S_{x} & S_{y} & S_{z} & 0\\H_{x} & H_{y} & H_{z} & 0\\- F_{x} & - F_{y} & - F_{z} & 0\\0 & 0 & 0 & 1\end{matrix}\right]$$

#### 視野変換 (LookAt行列)

平行移動行列と回転行列を合成することで視野変換行列が得られます．$$\mathrm {LookAt} = \mathrm {LookAtRotate} \cdot \mathrm {LookAtTranslate} =\left[\begin{matrix}S_{x} & S_{y} & S_{z} & - I_{x} S_{x} - I_{y} S_{y} - I_{z} S_{z}\\H_{x} & H_{y} & H_{z} & - H_{x} I_{x} - H_{y} I_{y} - H_{z} I_{z}\\- F_{x} & - F_{y} & - F_{z} & F_{x} I_{x} + F_{y} I_{y} + F_{z} I_{z}\\0 & 0 & 0 & 1\end{matrix}\right]$$

## 正射影 (Orthographic transformation)

正射影は空間から視線を中心線とする直方体領域を切り取り，それを原点を中心とした一辺の長さが2の立方体に射影します．直方体は視点から見たときの幅(*width*)と高さ(*height*)に加えて，直方体の手前と奥の面に対応する$Z$座標の値として，それぞれ$Z_{\mathit {near}}$と$Z_{\mathit {far}}$によって与えられます．

この変換は，直方体の中心座標($(0, 0, (Z_{\mathit {near}} + Z_{\mathit {far}}) / 2)$)を原点に移動する平行移動変換($T$)と直方体を立方体に拡縮を施す変換($S$)を合成したものと考えることができます．ここでの拡縮で視野座標系で用いていた左手系を右手系に戻します．この系の変更は，拡縮行列の計算にあたって$Z$軸についての拡縮率の符号を反転させている点に注意して下さい．

これらを合成することで，正射影を表す行列を$\mathrm {Ortho} = S \cdot T$と表現できます．

In [11]:
sp.var('width, height')
[near, far] = sp.symbols('Z_{near}, Z_{far}')

[tx, ty, tz] = TranslateSymbols

OrthoTranslate = Symbolic['Translate'].subs({ tx: 0, ty: 0, tz: (-(near + far)/2) })
OrthoScale = sp.diag(2/width, 2/height, -2/(far - near), 1)
Symbolic['Orthographic'] = OrthoScale * OrthoTranslate
Symbolic['orthographic'] = Symbolic['M32']((width, height, near, far), Symbolic['Orthographic'])

if __name__ == '__main__':
    md('#### 平行移動変換行列\n\n$',
        '\\mathrm {OrthoTranslate} =', OrthoTranslate, '$')

    md('平行移動変換によって直方体の中心$(0, 0, (near + far)/2)$が原点に移動することを確認してみましょう．')
    md('$$\\mathrm {OrthoTranslate}(\\mathit {near}, \\mathit {far})',
       Homogeneous(Vec3(0, 0, (near + far)/2)), '=',
       OrthoTranslate, '\\cdot', Homogeneous(Vec3(0, 0, (near + far)/2)), '=',
       OrthoTranslate * Homogeneous(Vec3(0, 0, (near + far)/2)), '$$')

    md('#### 拡大縮小行列\n\n$$',
       '\mathrm {OrthoScale} =', OrthoScale, '$$')

    md('これらの行列を合成することで正射影を表す$\mathrm {Orthographic}$行列を定義できます．')


    md('$$\\mathrm {Orthographic} = \\mathrm {OrthoScale} \cdot \\mathrm {OrthoTranslate} =',
       Symbolic['Orthographic'], '$$')

    [p1, p2] = [Homogeneous(p) for p in [Vec3( width/2,  height/2, near),
                                         Vec3(-width/2, -height/2, far)]]

    md('では，正射影によって直方体の対角線が立方体の対角線に射影されることを確認してみましょう．$$',
       '\\mathrm {Orthographic} \cdot', Homogeneous(Vec3(width/2, height/2, near)), '=',
       sp.simplify(Cartesian(Symbolic['Orthographic'] * p1)), '\\qquad',
   
       '\\mathrm {Orthographic} \cdot', Homogeneous(Vec3(-width/2, -height/2, far)), '=',
       sp.simplify(Cartesian(Symbolic['Orthographic'] * p2)), '$$')

#### 平行移動変換行列

$\mathrm {OrthoTranslate} =\left[\begin{matrix}1 & 0 & 0 & 0\\0 & 1 & 0 & 0\\0 & 0 & 1 & - \frac{Z_{{far}}}{2} - \frac{Z_{{near}}}{2}\\0 & 0 & 0 & 1\end{matrix}\right]$

平行移動変換によって直方体の中心$(0, 0, (near + far)/2)$が原点に移動することを確認してみましょう．

$$\mathrm {OrthoTranslate}(\mathit {near}, \mathit {far})\left[\begin{matrix}0\\0\\\frac{Z_{{far}}}{2} + \frac{Z_{{near}}}{2}\\1\end{matrix}\right]=\left[\begin{matrix}1 & 0 & 0 & 0\\0 & 1 & 0 & 0\\0 & 0 & 1 & - \frac{Z_{{far}}}{2} - \frac{Z_{{near}}}{2}\\0 & 0 & 0 & 1\end{matrix}\right]\cdot\left[\begin{matrix}0\\0\\\frac{Z_{{far}}}{2} + \frac{Z_{{near}}}{2}\\1\end{matrix}\right]=\left[\begin{matrix}0\\0\\0\\1\end{matrix}\right]$$

#### 拡大縮小行列

$$\mathrm {OrthoScale} =\left[\begin{matrix}\frac{2}{width} & 0 & 0 & 0\\0 & \frac{2}{height} & 0 & 0\\0 & 0 & - \frac{2}{Z_{{far}} - Z_{{near}}} & 0\\0 & 0 & 0 & 1\end{matrix}\right]$$

これらの行列を合成することで正射影を表す$\mathrm {Orthographic}$行列を定義できます．

$$\mathrm {Orthographic} = \mathrm {OrthoScale} \cdot \mathrm {OrthoTranslate} =\left[\begin{matrix}\frac{2}{width} & 0 & 0 & 0\\0 & \frac{2}{height} & 0 & 0\\0 & 0 & - \frac{2}{Z_{{far}} - Z_{{near}}} & - \frac{- Z_{{far}} - Z_{{near}}}{Z_{{far}} - Z_{{near}}}\\0 & 0 & 0 & 1\end{matrix}\right]$$

では，正射影によって直方体の対角線が立方体の対角線に射影されることを確認してみましょう．$$\mathrm {Orthographic} \cdot\left[\begin{matrix}\frac{width}{2}\\\frac{height}{2}\\Z_{{near}}\\1\end{matrix}\right]=\left[\begin{matrix}1\\1\\1\end{matrix}\right]\qquad\mathrm {Orthographic} \cdot\left[\begin{matrix}- \frac{width}{2}\\- \frac{height}{2}\\Z_{{far}}\\1\end{matrix}\right]=\left[\begin{matrix}-1\\-1\\-1\end{matrix}\right]$$

## 錐台変換？ (Frustum transformation)

正射影との違いを理解していない．．．式は異なるようだが，何をしている？？？

In [12]:
def frustum(left, right, bottom, top, near, far):
    rl, tb, fn = right - left, top - bottom, far - near
    return mat4x4(
            2 * near / rl, 0, (right + left) / rl, 0,
            0, 2 * near / tb, (top + bottom) / tb, 0,
            0, 0, -(far + near) / fn, -2 * far * near / fn,
            0, 0, 0, 0)

''' GLM implementation
template <typename T> GLM_FUNC_QUALIFIER tmat4x4<T, defaultp>
frustum (T left, T right, T bottom, T top, T nearVal, T farVal) {
    tmat4x4<T, defaultp> Result(0);
    Result[0][0] = (static_cast<T>(2) * nearVal) / (right - left);
    Result[1][1] = (static_cast<T>(2) * nearVal) / (top - bottom);
    Result[2][0] = (right + left) / (right - left);
    Result[2][1] = (top + bottom) / (top - bottom);
    Result[2][2] = -(farVal + nearVal) / (farVal - nearVal);
    Result[2][3] = static_cast<T>(-1);
    Result[3][2] = -(static_cast<T>(2) * farVal * nearVal) / (farVal - nearVal);
    return Result;
}
'''

if __name__ == '__main__':
    print('Frustum(0, 10, 0, 10, 2, 12):\n{0}'.format(frustum(0, 10, 0, 10, 2, 12)))

Frustum(0, 10, 0, 10, 2, 12):
[[ 0.40000001  0.          1.          0.        ]
 [ 0.          0.40000001  1.          0.        ]
 [ 0.          0.         -1.39999998 -4.80000019]
 [ 0.          0.          0.          0.        ]]


## 透視投影 (Perspective transformation)

透視投影は，視野空間において視点から視る視野錐台(*view frustum*)をクリッピング空間上の立方体に射影する変換で，遠近法を表現します．透視投影は視野角(*fovy*; field of view $y$)，アスペクト比($\mathit {aspect} = \mathit {width} / \mathit {height}$)，そして正射影と同様に奥行方向に切り取るための*near*と*far*を用います．視野角はラジアンによって左右方向の視野を定めます．この視野角とアスペクト比によって上下方向の視野角が定まります．

錐台の手前の面($z = \mathit{near}$)の四隅の点
$(\pm \tan(\mathit {fovy}) \times \mathit {aspect}, \pm \mathit {fovy}, -1) \times \mathit {near}$はそれぞれ透視投影によって$(\pm 1, \pm 1, -1)$で表現される点に写され，錐台の遠方の面($z = \mathit{far}$)の四隅の点
$(\pm \tan(\mathit {fovy}), \pm \tan(\mathit {fovy}) \times \mathit {aspect}, -1) \times \mathit {far}$はそれぞれ透視投影によって$(\pm 1, \pm 1, 1)$に写されます．

以上を整理すると，透視投影変換によって８点が以下のように変換されます．
$$\begin {align*}
(\pm \tan(\mathit {fovy}), \pm \tan(\mathit {aspect}) \times \mathit {fovy}, １) \times \mathit {near} & \mapsto (\pm 1, \pm 1, ー1) \\
(\pm \tan(\mathit {fovy}), \pm \tan(\mathit {aspect}) \times \mathit {fovy}, １) \times \mathit {far} & \mapsto (\pm 1, \pm 1, 1) \\
\end {align*}$$

この射影から線形独立な４点を選択して行列として表現したものを$P$として、それらの点が透視投影変換行列$T$で、$P'$に写るさまを$TP = P'$と表現できます．これより透視投影変換行列は$T = P' P^{-1}$となります。

In [109]:
PerspectiveSymbols = sp.var('fovy aspect near far')
ny = near * sp.tan(fovy / 2)
nx = ny * aspect
fy = far * sp.tan(fovy / 2)

PointsCamera = sp.Matrix([Homogeneous(Vec3(*v)).T
                          for v in [(0, 0, -near), (nx, 0, -near), (0, ny, -near),
                                    (0, fy, -far)]]).T

PointsPerspective = sp.Matrix([Homogeneous(Vec3(*v), w=near).T
                               for v in [(0, 0, -1), (1, 0, -1), (0, 1, -1)]]).T
PointsPerspective = PointsPerspective.row_join(Homogeneous(Vec3(0, 0, 1), w=far))

Symbolic['Perspective'] = sp.simplify(PointsPerspective * PointsCamera.inv())
Symbolic['perspective'] = Symbolic['M32'](PerspectiveSymbols, Symbolic['Perspective'])

if __name__ == '__main__':
    md('## Perspective変換行列\n', '$$', Symbolic['Perspective'], '$$')
    # デカルト座標系において視野錐台が確かに、単位立方体に射影されることの確認
    Points = sp.simplify(Symbolic['Perspective'] * PointsCamera)
    for c in range(4):
        assert(Cartesian(Points[:,c]) == Cartesian(PointsPerspective[:,c]))

## Perspective変換行列
$$\left[\begin{matrix}\frac{1}{aspect \tan{\left (\frac{fovy}{2} \right )}} & 0 & 0 & 0\\0 & \frac{1}{\tan{\left (\frac{fovy}{2} \right )}} & \frac{far}{far - near} & \frac{far near}{far - near}\\0 & 0 & - \frac{far + near}{far - near} & - \frac{2 far near}{far - near}\\0 & 0 & -1 & 0\end{matrix}\right]$$

In [16]:
def perspectiveFov(fov, width, height, near, far):
    h = cos(fov/2) / sin(fov/2)
    w = h * height / width
    return mat4x4(
            w, 0, 0, 0,
            0, h, 0, 0,
            0, 0, -(far + near) / (far - near), -1, 
            0, 0, -2 * far * near / (far - near))

'''GLM implementation
template <typename T> GLM_FUNC_QUALIFIER tmat4x4<T, defaultp>
perspectiveFov (T fov, T width, T height, T zNear, T zFar) {
    assert(width > static_cast<T>(0));
    assert(height > static_cast<T>(0));
    assert(fov > static_cast<T>(0));

    T const rad = fov;
    T const h = glm::cos(static_cast<T>(0.5) * rad) / glm::sin(static_cast<T>(0.5) * rad);
    T const w = h * height / width; ///todo max(width , Height) / min(width , Height)?

    tmat4x4<T, defaultp> Result(static_cast<T>(0));
    Result[0][0] = w;
    Result[1][1] = h;
    Result[2][2] = - (zFar + zNear) / (zFar - zNear);
    Result[2][3] = - static_cast<T>(1);
    Result[3][2] = - (static_cast<T>(2) * zFar * zNear) / (zFar - zNear);
    return Result;
}'''

if __name__ == '__main__':
    pass

In [17]:
def project(obj, Model, Proj, viewport):
    V = Proj.dot(Model.dot(obj))
    V = V / V[3] / 2 + 0.5
    V[0] = V[0] * viewport[2] + viewport[0]
    V[1] = V[1] * viewport[3] + viewport[1]
    return V

'''GLM implementation
template <typename T, typename U, precision P> GLM_FUNC_QUALIFIER tvec3<T, P>
project (tvec3<T, P> const & obj, tmat4x4<T, P> const & model, tmat4x4<T, P> const & proj, tvec4<U, P> const & viewport) {
    tvec4<T, P> tmp = tvec4<T, P>(obj, T(1));
    tmp = model * tmp;
    tmp = proj * tmp;

    tmp /= tmp.w;
    tmp = tmp * T(0.5) + T(0.5);
    tmp[0] = tmp[0] * T(viewport[2]) + T(viewport[0]);
    tmp[1] = tmp[1] * T(viewport[3]) + T(viewport[1]);

    return tvec3<T, P>(tmp);
}'''

if __name__ == '__main__':
#    print('Project(obj, Model, Proj, viewport):\n{0}'.format(perspective(np.pi / 3, 1, 2, 12)))
    pass